# Task 1: Loading the dataset

In [1]:
import pandas as pd
df = pd.read_csv('IMDB Dataset.csv')

In [2]:
#After loading, lets introspect this dataset
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [3]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

# Task 2: Transforming Docs into Feature Vectors

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#initialization of count vectorizer
count = CountVectorizer() 
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
 #tranform the word into vector main role here
bag = count.fit_transform(docs)

In [5]:
#unique words in vocab and counts the words
print(count.vocabulary_) 

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [6]:
#3 features array for each different 3 docs
print(bag.toarray()) 

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Task 3: Term Frequency & Inverse DOC frequency

In [7]:
#Term frequencies alone do not contribute to distinct information
#tf-idf (t,d) = tf(t,d) X idf(t,d)
#idf(t,d) = log(nd/1 + df(d,t)) #log is nullifying 
#nd = total number of docs
#df(d,t) = no of docs that contain the term t

In [8]:
#task 4
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)

In [9]:
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [10]:
#task 5
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stemmer_tokenize(text):
    return [porter.stem(word) for word in text.split()]

stemmer_tokenize('Coders like coding and thus they code')

['coder', 'like', 'code', 'and', 'thu', 'they', 'code']

In [11]:
#Task 6 Vectorizing the dataset
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,
                       lowercase=False,
                       tokenizer = stemmer_tokenize,
                       use_idf = True,
                       norm = 'l2',
                       smooth_idf=True)
Y = df.sentiment.values
X = tfidf.fit_transform(df.review)

# Task 7: Doc classification using Logistic Regression

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 1, test_size = 0.5, shuffle = False)

import pickle
from sklearn.linear_model import LogisticRegressionCV

In [13]:
#model

clf = LogisticRegressionCV(cv = 5,
                          scoring = 'accuracy',
                          random_state = 0,
                          n_jobs = -3,
                          verbose = 3,
                          max_iter = 300).fit(X_train, Y_train)

[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done   2 out of   5 | elapsed: 15.0min remaining: 22.5min
[Parallel(n_jobs=-3)]: Done   5 out of   5 | elapsed: 16.3min finished


In [14]:
#saving model

saved_model = open('saved_model.sav','wb') #file we create we write a dump file

#using the pickle library's dump function to write the trained classifier
pickle.dump(clf, saved_model)
saved_model.close()

# Task 8: Model Evaluation

In [15]:
filename = 'saved_model.sav'

saved_clf = pickle.load(open(filename, 'rb'))

#test the saved model on test data
saved_clf.score(X_test, Y_test)

C:\Users\pjyaw\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.88988